In [ ]:
import tensorflow as tf
import numpy as np

from pinn import PINNModel, model1
from accuracy import AccuracyCalc
from plotting import NNPlots
from data_gen import DataGenerator

In [2]:
def real_u(area):
    area = np.array(area)
    if len(area.shape) >= 3:
        x = area[0]
        y = area[1]
        return np.sin(np.pi * x) * np.sin(np.pi * y)
    else:
        return np.array([np.sin(np.pi * x) * np.sin(np.pi * y)  for x, y in area])

In [3]:
# force gpu use
gpus = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

In [ ]:
area_grid_size = (50, 50)
x_limits = (0, 2)
y_limits = (0, 2)
border_grid_size = (50, 50)

EPOCHS = 6000
EPRINT = 500
LOSS = 1e-4

model = PINNModel(model1(4), optm = tf.keras.optimizers.Adam(learning_rate=0.001))

dataGen = DataGenerator(x_limits, y_limits, model.predict, real_u)

In [ ]:
model.train(3, dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size), EPOCHS, LOSS, EPRINT)

In [6]:
# for k in range(1, 10):
#     # model.reset_weights()
#     model.train(k, dataGen.inner_pairs(area_grid_size), dataGen.border_pairs(border_grid_size), EPOCHS, LOSS, EPRINT)
    
#     print("Error:", acc.mse())
#     print("Error:", acc.maxe())

#     model.save(f"../models/model{k}.weights.h5")

#     break

In [ ]:
# Should be created after training model, because of predict in __init__
plotter = NNPlots(dataGen, (200, 200))
acc = AccuracyCalc(dataGen, (70, 70))

In [ ]:
print("Error:", acc.mse())
print("Error:", acc.maxe())

In [ ]:
plotter.plot2d_contour(color_map="gnuplot2")